# Bank Customer Segmentation 

In [1]:
#importing libraries
import numpy as np
import pandas as pd 
import warnings

In [2]:
#reading file
df=pd.read_csv("Desktop/data/bank_transactions.csv")
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


# Data Cleaning

In [3]:
#droping transaction_ID
df=df.drop(['TransactionID'],axis=1)

In [4]:
df.columns

Index(['CustomerID', 'CustomerDOB', 'CustGender', 'CustLocation',
       'CustAccountBalance', 'TransactionDate', 'TransactionTime',
       'TransactionAmount (INR)'],
      dtype='object')

In [5]:
df.describe()

,CustAccountBalance,TransactionTime,TransactionAmount (INR)
count,1.046198e+06,1.048567e+06,1.048567e+06
mean,1.154035e+05,1.570875e+05,1.574335e+03
std,8.464854e+05,5.126185e+04,6.574743e+03
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.721760e+03,1.240300e+05,1.610000e+02
50%,1.679218e+04,1.642260e+05,4.590300e+02
75%,5.765736e+04,2.000100e+05,1.200000e+03
max,1.150355e+08,2.359590e+05,1.560035e+06


# Appending the locations with Alternate Names

In [19]:
print("Location:")
Location = df['CustLocation'].value_counts().head(30)
print(Location)

df['CustLocation'][df['CustLocation'] == 'NEW DELHI'] = 'DELHI'
df['CustLocation'][df['CustLocation'] == 'NAVI MUMBAI']='MUMBAI'

warnings.filterwarnings("ignore")

Location:
DELHI            143715
MUMBAI           109611
BANGALORE         77751
GURGAON           70518
NOIDA             31675
CHENNAI           27500
PUNE              24493
HYDERABAD         21819
THANE             21006
KOLKATA           19033
GHAZIABAD         14624
AHMEDABAD         11266
FARIDABAD         10641
JAIPUR             9231
CHANDIGARH         8801
LUCKNOW            7338
MOHALI             5828
SURAT              5173
NASHIK             4645
LUDHIANA           4354
VISAKHAPATNAM      4173
DEHRADUN           3961
INDORE             3620
VADODARA           3584
AMRITSAR           3548
AGRA               3519
KANPUR             3326
NAGPUR             3312
MEERUT             3247
COIMBATORE         3183
Name: CustLocation, dtype: int64


In [7]:
#checking for N/A values
df.isna().sum()

CustomerID                    0
CustomerDOB                3397
CustGender                 1100
CustLocation                151
CustAccountBalance         2369
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
dtype: int64

In [8]:
#droping N/A values
df = df.dropna()
df.isna().sum()

CustomerID                 0
CustomerDOB                0
CustGender                 0
CustLocation               0
CustAccountBalance         0
TransactionDate            0
TransactionTime            0
TransactionAmount (INR)    0
dtype: int64

In [9]:
#to find null values
df.isnull().sum()

CustomerID                 0
CustomerDOB                0
CustGender                 0
CustLocation               0
CustAccountBalance         0
TransactionDate            0
TransactionTime            0
TransactionAmount (INR)    0
dtype: int64

In [10]:
#to find abnormal pattern in dates
df['CustomerDOB'].value_counts()

1/1/1800    56292
1/1/89        809
1/1/90        784
6/8/91        698
1/1/91        665
            ...  
5/1/51          1
2/10/98         1
20/12/53        1
11/12/58        1
4/10/69         1
Name: CustomerDOB, Length: 17233, dtype: int64

Since there 56292 whose DOB are registered as 1800-01-01, we have a total data of 1Million plus bank transactions so we will discard them

In [11]:
#Removal of dates which are absurd.
df = df.loc[~(df['CustomerDOB'] == '1/1/1800')]
df['CustomerDOB'].value_counts()

1/1/89     809
1/1/90     784
6/8/91     698
1/1/91     665
1/1/92     631
          ... 
22/2/61      1
8/11/96      1
19/8/55      1
28/3/15      1
26/3/35      1
Name: CustomerDOB, Length: 17232, dtype: int64

# updating date formats

In [12]:
#assigning proper format
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], format = '%d/%m/%y')
df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'])

In [13]:
df.head()

,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-08-02,143207,25.0
1,C2142763,2057-04-04,M,JHAJJAR,2270.69,2016-08-02,141858,27999.0
2,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,142712,459.0
3,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,142714,2060.0
4,C9031234,1988-03-24,F,MUMBAI,6714.43,2016-08-02,181156,1762.5


# Customer age during the transaction

In [14]:
# conversion of years greater than 2021 
df.loc[df['CustomerDOB'].dt.year >= 2021, ['CustomerDOB']] -= pd.DateOffset(years = 100)
#calculation
df['CoustomerAge'] = (pd.to_datetime('today') - df['CustomerDOB'])/np.timedelta64(1, 'Y')
df.head()

,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CoustomerAge
0,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-08-02,143207,25.0,27.242172
1,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-08-02,141858,27999.0,64.735070
2,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,142712,459.0,25.087439
3,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,142714,2060.0,48.288463
4,C9031234,1988-03-24,F,MUMBAI,6714.43,2016-08-02,181156,1762.5,33.763866


In [15]:
#renaming the column
df = df.rename(columns = {'TransactionAmount (INR)' : 'TransactionAmount'})

In [16]:
#convertig age to int type from float.
df=df.astype({'CoustomerAge':int})
df.head()

,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount,CoustomerAge
0,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-08-02,143207,25.0,27
1,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-08-02,141858,27999.0,64
2,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,142712,459.0,25
3,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,142714,2060.0,48
4,C9031234,1988-03-24,F,MUMBAI,6714.43,2016-08-02,181156,1762.5,33


In [17]:
df['TransactionYear'] = pd.to_datetime(df['TransactionDate'], format = '%y')
df.head()

,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount,CoustomerAge,TransactionYear
0,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-08-02,143207,25.0,27,2016-08-02
1,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-08-02,141858,27999.0,64,2016-08-02
2,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,142712,459.0,25,2016-08-02
3,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,142714,2060.0,48,2016-08-02
4,C9031234,1988-03-24,F,MUMBAI,6714.43,2016-08-02,181156,1762.5,33,2016-08-02


In [18]:
#extracting year
df['TransactionYear'] = df['TransactionDate'].dt.year
df['TransactionMonth'] = df['TransactionDate'].dt.month
df.head()

,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount,CoustomerAge,TransactionYear,TransactionMonth
0,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-08-02,143207,25.0,27,2016,8
1,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-08-02,141858,27999.0,64,2016,8
2,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,142712,459.0,25,2016,8
3,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,142714,2060.0,48,2016,8
4,C9031234,1988-03-24,F,MUMBAI,6714.43,2016-08-02,181156,1762.5,33,2016,8
